In [152]:
# Load necessary modules
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import io

By experimenting with the side we can see that there are 190 pages of results in total. Thus, we should load a given page, extract all the addresses with community-sourced churches, and append them to a dataframe. 

In [147]:
BaseURL = "https://www.catholicdirectory.com/search_results?country_sn=FR&location_type=country&swlat=41.31433&nelat=51.1241999&swlng=-5.5591&nelng=9.6624999&lat=46.227638&lng=2.213749&faddress=France&place_id=ChIJMVd4MymgVA0R99lHx5Y__Ws&location_value=france+Europe&page={}"

In [204]:
df = pd.DataFrame()

for page in range(1, 191):
    URL = BaseURL.format(page)
    print(page)
    
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser').text
    cleanedsoup = soup.replace('\n', '').replace('\t', '')
    cleanedsoup = io.StringIO(cleanedsoup)

    cleanedsoup_ds = pd.read_csv(cleanedsoup,
                                 header=None,
                                 delimiter='\r',
                                 skipinitialspace=True,
                                 error_bad_lines=False,
                                 lineterminator='\r')
    
    # Extract only those that have a character in there
    cleanedsoup_ds = cleanedsoup_ds[cleanedsoup_ds[0].str.contains(' View ')]
    
    # Append to df
    df = pd.concat([df, cleanedsoup_ds], sort = False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


Clean up the column a bit more by stripping unnecessary text. 

In [209]:
# Rename column
df.columns = ['Address']

,Address
5,"Herimoncourt, Bourgogne Franche-Comté, 25310 V..."
7,"Belval, Normandy, 62130 View ListingLIKE"
9,"Bayeux, Normandy, 14400 View ListingLIKE"
11,"Wisques, 62219 View ListingLIKE"
13,"Saint Wandrille Rancon, Normandie, 76490 View ..."


In [211]:
# Remove excess html text
df['Address'] = df['Address'].str.replace(' View ListingLIKE', '')

In [212]:
df.head()

,Address
5,"Herimoncourt, Bourgogne Franche-Comté, 25310"
7,"Belval, Normandy, 62130"
9,"Bayeux, Normandy, 14400"
11,"Wisques, 62219"
13,"Saint Wandrille Rancon, Normandie, 76490"


Save the dataset for future geocoding.

In [215]:
df.to_csv('../data/to_scrape/CatholicDirectoryScrape.csv')